In [93]:
# Esports Statistics from 1998-2024
import pandas as pd
import matplotlib.pyplot as plt
import numpy

In [94]:
data = pd.read_csv("HistoricalEsportData.csv")


data["Earnings"] = pd.to_numeric(data["Earnings"], errors="coerce")
data["Players"] = pd.to_numeric(data["Players"], errors="coerce")
data["Tournaments"] = pd.to_numeric(data["Tournaments"], errors="coerce")

data["Date"] = pd.to_datetime(data["Date"], errors="coerce")

data["Game_numerical"] = pd.factorize(data["Game"])[0]






In [95]:
ge_data = pd.read_csv("GeneralEsportData.csv")

ge_data["Genre_number"] = pd.factorize(ge_data["Genre"])[0]




In [96]:

data["Genre"] = data["Game"].map(ge_data.set_index("Game")["Genre"])

print(data)

data.to_csv("new_data.csv", index = False)


            Date                                Game  Earnings  Players  \
0     1998-01-01        Command & Conquer: Red Alert   15000.0        8   
1     1998-01-01                          QuakeWorld   15000.0        8   
2     1998-05-01                            Quake II   15000.0        4   
3     1998-05-01                  Total Annihilation   15000.0        2   
4     1998-05-01                          QuakeWorld     300.0        3   
...          ...                                 ...       ...      ...   
10234 2024-10-01       Super Street Fighter II Turbo     340.0        3   
10235 2024-10-01               Skullgirls 2nd Encore     330.0        3   
10236 2024-10-01       StarCraft: Evo Complete (Mod)     308.0        4   
10237 2024-10-01            Melty Blood: Type Lumina     280.0        3   
10238 2024-10-01  Power Rangers: Battle for the Grid     200.0        3   

       Tournaments  Game_numerical                 Genre  
0                1               0      